Team HKV: Michael Keller, Henry Herzfeld, Yuri Villanueva

In this report, we attempt to implement RSA in Python. While the C library NTL is preferred by many for large number operations, we chose to invesigate whether Python can achieve the same or better performance than NTL using existing libraries or packages in conjunction with those that we create. Our results will be detailed in this document, in the hopes that they can be compared with those of other groups in order to make this determination.

# Naive RSA Decryption vs CRT Decryption

First, we were asked to investigate the difference in performance between a naive RSA implementation and an RSA implementation that uses the Chinese Remainder Theorem. 

First, we must perform key generation. We will create key sizes that are powers of 2 up to 8192 bits for comparison.

In [1]:
from RSA import RSA
from powmodn import rec_pow_mod_n, bit_pow_mod_n
from util import lcm, int2string, string2int
from math import gcd
from Crypto.Util import number
from rt import rt2, rt4, rt_average_2, rt_average_4
from inverse import rec_inverse

In [2]:
scheme = RSA(powmodn=bit_pow_mod_n)

(p1, q1, n1, l1, e1, d1, public_key1, private_key1) = scheme.generate_keys(bit_length=1024)
(p2, q2, n2, l2, e2, d2, public_key2, private_key2) = scheme.generate_keys(bit_length=2048)
(p3, q3, n3, l3, e3, d3, public_key3, private_key3) = scheme.generate_keys(bit_length=4096)

RSA scheme using bit_pow_mod_n and rec_inverse algorithms


Next, we perform encryption with each bit size:

In [3]:
message="The quick brown fox jumps over the lazy dog."
print("\nOriginal plaintext message: ", message)

m = string2int(message)

c1 = scheme.rsa_encrypt(m, public_key1)
ciphertext1 = int2string(c1)

c2 = scheme.rsa_encrypt(m, public_key2)
ciphertext2 = int2string(c2)

c3 = scheme.rsa_encrypt(m, public_key3)
ciphertext3 = int2string(c3)


Original plaintext message:  The quick brown fox jumps over the lazy dog.


Now we run the two decryption algorithms with the three different bit sizes and compare their running times:

In [6]:
print ("1024 bits: \n")

m2_1024 = scheme.rsa_decrypt(c1, private_key1)
running_time_avg_1024 = rt_average_2(scheme.rsa_decrypt, c1, private_key1, 10)
message2_1024 = int2string(m2_1024)
print(" Message decrypted by rsa_decrypt: ", message2_1024)
print(" Average Running time over 10 runs for rsa_decrypt: ", running_time_avg_1024)

m3_1024 = scheme.CRT_rsa_decrypt(c1, private_key1, p1, q1)
CRT_running_time_avg_1024 = rt_average_4(scheme.CRT_rsa_decrypt, c1, private_key1, p1, q1, 10)
message3_1024 = int2string(m3_1024)
print(" Message decrypted by CRT_rsa_decrypt (seconds): ", message3_1024)
print(" Average Running time for 10 runs of CRT_rsa_decrypt (seconds): ", CRT_running_time_avg_1024)

print ("2048 bits: \n")

m2_2048 = scheme.rsa_decrypt(c2, private_key2)
running_time_avg_2048 = rt_average_2(scheme.rsa_decrypt, c2, private_key2, 10)
message2_2048 = int2string(m2_2048)
print(" Message decrypted by rsa_decrypt: ", message2_2048)
print(" Average Running time over 10 runs for rsa_decrypt: ", running_time_avg_2048)

m3_2048 = scheme.CRT_rsa_decrypt(c2, private_key2, p2, q2)
CRT_running_time_avg_2048 = rt_average_4(scheme.CRT_rsa_decrypt, c2, private_key2, p2, q2, 10)
message3_2048 = int2string(m3_2048)
print(" Message decrypted by CRT_rsa_decrypt (seconds): ", message3_2048)
print(" Average Running time for 10 runs of CRT_rsa_decrypt (seconds): ", CRT_running_time_avg_2048)

print ("4096 bits: \n")

m2_4096 = scheme.rsa_decrypt(c3, private_key3)
running_time_avg_4096 = rt_average_2(scheme.rsa_decrypt, c3, private_key3, 10)
message2_4096 = int2string(m2_4096)
print(" Message decrypted by rsa_decrypt: ", message2_4096)
print(" Average Running time over 10 runs for rsa_decrypt: ", running_time_avg_4096)

m3_4096 = scheme.CRT_rsa_decrypt(c3, private_key3, p3, q3)
CRT_running_time_avg_4096 = rt_average_4(scheme.CRT_rsa_decrypt, c3, private_key3, p3, q3, 10)
message3_4096 = int2string(m3_4096)
print(" Message decrypted by CRT_rsa_decrypt (seconds): ", message3_4096)
print(" Average Running time for 10 runs of CRT_rsa_decrypt (seconds): ", CRT_running_time_avg_4096)

1024 bits: 

 Message decrypted by rsa_decrypt:  b'The quick brown fox jumps over the lazy dog.'
 Average Running time over 10 runs for rsa_decrypt:  0.028283071517944337
 Message decrypted by CRT_rsa_decrypt (seconds):  b'The quick brown fox jumps over the lazy dog.'
 Average Running time for 10 runs of CRT_rsa_decrypt (seconds):  0.009477663040161132
2048 bits: 

 Message decrypted by rsa_decrypt:  b'The quick brown fox jumps over the lazy dog.'
 Average Running time over 10 runs for rsa_decrypt:  0.19391233921051027
 Message decrypted by CRT_rsa_decrypt (seconds):  b'The quick brown fox jumps over the lazy dog.'
 Average Running time for 10 runs of CRT_rsa_decrypt (seconds):  0.058762431144714355
4096 bits: 

 Message decrypted by rsa_decrypt:  b'The quick brown fox jumps over the lazy dog.'
 Average Running time over 10 runs for rsa_decrypt:  1.3401647329330444
 Message decrypted by CRT_rsa_decrypt (seconds):  b'The quick brown fox jumps over the lazy dog.'
 Average Running time fo

## The Random Fault Attack

It was also requested that we implement the random fault attack on the signature of RSA given in the following paper:
https://crypto.stanford.edu/~dabo/pubs/papers/RSA-survey.pdf 

This attack proceeds as follows:

Alice asks Bob to sign a message $m$. Normally, Bob signs the message with his private key $sk$. Alice can verify this signature $s$ using the public key $pk$. It is the same as the original message $m$. 

The random fault attack works on the CRT implementation of RSA decryption. With CRT, RSA signing/decrypting first sends the message $m$ in $Z_n$ to the corresponding element $(u, v)$ in $Z_p x Z_q$. Exponentiation is done in $Z_p x Z_q$ where it is less computationally expensive:

$$x= u^d (mod (p-1))$$

$$w=v^d (mod (q-1))$$

Afterwards, send $(x, w)$ in $Z_p x Z_q$ to the corresponding element in $Z_n$. This can be done using the Extended Euclidean Algorithm, where we find $r$ and $t$, the inverses of $p$ in $Z_q$, and $q$ in $Z_p$, respectively, so that:

$$rpx + tqw = w (mod p) = x (mod q)$$

is the corresponding element in $Z_n$. This is the signature $s = m ^ d (mod n)$.

Note that $spx = 0 (mod p)$ and $tqw = 0 (mod q)$. The random fault attack can be mounted if exactly one of $w$ or $x$ can be corrupted.

Suppose exactly one of $x$ or $w$ has bit errors, say $x$ becomes $x'$. Then Alice doesn't get the original message $m$ when she verifies the signature. Instead of $s^e (mod n) = m$, Alice gets: 

$$m' = (rpx' + tqw)^e (mod n)= (rpx')^e + (tpq)^e (mod n)$$

The difference between this and $m = s^d = (rpx) ^ e + (tpq) ^ e (mod n)$ is $(rpx') ^ e - (rpx)^e (mod n)$. This difference is equivalent to $0 (mod p)$, and just as important, this difference is nonzero. That is, $m - verify(s', public_key) = 0 (mod p)$ and is nonzero. Thus, computing $gcd(n, m - verify(s' public_key))$ reveals one of the factors of n. When we introduce even a single bit error in $x$ or $w$, we don't get the original message $m$ when we verify the corrupted signature.

In our implementation, the CRT_rsa_sign function has a 'feature' that lets us introduce a bit error at a random position in either $x$ or $w$. Alice won't get the original message back when she verifies $s$'. The difference between this corrupted message and the original message is equivalent to 0 modulo one of the prime factors of $n$. Alice can recover the factorization by computing the gcd. This factorization should be equal to one of the prime factors of n.

To demonstrate this, we present the following code. We use 1024 bits here for ease of computation.

In [ ]:
scheme = RSA(powmodn=bit_pow_mod_n, sign=True)

bit_length = 1024
e = 65537

print("\n\n-------------------------------------------------------------")
print("\nThe random fault attack:")
print("Generating", bit_length, "-bit keys...")

(p, q, n, l, e, d, public_key, private_key) = scheme.generate_keys(bit_length, e)

message = "Please attach your signature."
print("\n Message to be signed:\n\n\t", message)
m = string2int(message)

s = scheme.rsa_sign(m, private_key, p, q)
print("\n Signature creation successful!")

s1 = scheme.rsa_sign(m, private_key, p, q, faulty=True)  # random bit flip

print("\n Created corrupted signature \n")

m1 = scheme.rsa_verify(s1, public_key)

print("Are m and m' the same? \t", m1==m)

recovered_factor = gcd(n, m - m1)

print("Is recovered_factor=p?\t", recovered_factor==p)
print("Is recovered_factor=q?\t", recovered_factor==q)

We note here that $gcd(n, m - m')=p$, verifying the correctness of our fault attack implementation.

## The Timing Attack

In [ ]:
##To be implemented